# Understanding Event Data

## Introduction

Neutron-scattering data may be recorded in "event mode":
For each detected neutron a (pulse) timestamp and a time-of-flight is stored.
This notebook will develop an understanding of how do work with this type of data.

Our objective is *not* to demonstrate or develop a full reduction workflow.
Instead we *develop understanding of data structures and opportunities* that event data provides.

This tutorial contains exercises, but solutions are included directly.
We encourage you to download this notebook and run through it step by step before looking at the solutions.
We recommend to use a recent version of *JupyterLab*:
The solutions are included as hidden cells and shown only on demand.

We use data containing event data form the POWGEN powder diffractometer at SNS.
Note that the data has been modified for the purpose of this tutorial and is not entirely in its original state.
We begin by loading thr file and plot the raw data:

In [ ]:
import scipp as sc
run = 4844
filename = f'PG3_{run}_event.h5'
da = sc.io.open_hdf5(filename=filename)
da.plot()

We can see some traces of diffraction lines, but they are oddly blurry.
There is also an artifact from the prompt-pulse visiable at $4000 \mu s$.
This tutorial illustrates how event data gives us the power to get to deal with and understand the underlying issues.
Before we start the investigation we cover some basics of working with event data.

## Inspecting event data

As usual, to begin exploring a loaded file, we can inspect the HTML representation of a scipp object shown by Jupyter when typing a variable at the end of a cell (this can also be done using `sc.to_html(da)`, anywhere in a cell):

In [ ]:
da

When can tell that this is binned (event) data from the `dtype` of the data (usually `DataArrayView`) as well as the inline preview, denoting that this is binned data with lists of given lengths.
The meaning of these can best be understood using a graphical depection of `da`, creating using `sc.show`:

In [ ]:
sc.show(da)

Each value (yellow cube) is a small table containing event parameters such as pulse time, time-of-flight, and weights (usually 1 for raw data).

**Definitions**:
1. In scipp we refer to each of these cubes (containing a table of events) as a *bin*.
   We can think of this as a bin (or bucket) containing a number of records.
2. An array of bins (such as the stack of yellow cubes above) is referred to as *binned variable*.
   For example, `da.data` is a binned variable.
3. A data array with data given by a binned variable is referred to as as *binned data*.
   Binned data is a precursor to dense or histogrammed data.

Each bin "contains" a small table, essentially a 1-D data array.
For efficiency and consistency scipp does not actually store an individual data array for every bin.
Instead each bin refers to a section (slice) of a long table containing all the events from all bins combined.
This explains the `dtype=DataArrayView` seen in the HTML representation above.
For many practical purposes such a view of a data arrays behaves just like any other data array.

The values of the bins can be accessed using the `values` property.
For dense data this might give us a `float` value, for binned data we obtain a table:

In [ ]:
da.values[500]

### Exercise

Use `sc.to_html()`, `sc.show()`, and `sc.table()` to explore and understand `da`, `da.events`, as well as individual values of `da` such as `da.values[500]`.

## From binned data to dense data

### Option 1: Summing bins

If the existing binning is sufficient for our purpose we may simply sum over the rows of the tables making up the bin values:

In [ ]:
da_bin_sum = da.bins.sum()

Here we used the special `bins` property to apply an operation to each of the bins in `da`.

We can visualize the result.
Make sure to compare the representations with those obtained above for binned data (`da`):

In [ ]:
sc.to_html(da_bin_sum)
sc.show(da_bin_sum)

We can use `da_bins_sum` to, e.g., plot the total counts per spectrum by summing over the `tof` dimension (in this case there is just a single time-of-flight bin so we could have used `da_bin_sum['tof', 0]` instead):

In [ ]:
da_bin_sum.sum('tof').plot(marker='.')

### Option 2: Histogramming

For performance and memory reasons binned data often contains the minimum number of bins that is "necessary" for a given purpose.
In this case `da` only contains a single time-of-flight bin, which is not practical for downstream applications such as data analysis, or plotting.
Instead of simply summing over all events in a bin we may thus *histogram* data.
Note that scipp makes the distinction between binning data (preserving all events individually) and histogramming data (summing all events that fall inside a bin).

For simplicity we consider only a single spectrum:

In [ ]:
spec = da['spectrum', 450].copy()
sc.show(spec)

In [ ]:
sc.table(spec.values[0]['event',:5])

We use one of the [scipp functions for creating a new variable](https://scipp.github.io/reference/api.html#creation-functions) to define the desired bin edge of our histogram.
In this case `sc.linspace`:

In [ ]:
tof_edges = sc.linspace(dim='tof', start=18.0, stop=17000, num=100, unit='us')
sc.histogram(spec, bins=tof_edges).plot()

#### Exercise

Change `tof_edges` to control what is plotted:

- Change the number of bins, e.g., to a finer resolution
- Change the start and stop of the edges to plot only a smaller time-of-flight region

#### Solution

In [ ]:
tof_edges = sc.linspace(dim='tof', start=2000.0, stop=15000, num=200, unit='us')
sc.histogram(spec, bins=tof_edges).plot()

## Masking event data — Binning by existing parameters

While quickly converting binned (event) data into dense (histogrammed) data has its applications, we may typically want to work with binned data as long as possible.
How can we mask a time-of-flight region, e.g., to mask a prompt-pulse, in *event mode*?

Let us sum all spectra and define a dummy data array (named `prompt`) to illustrate the objective:

In [ ]:
# Start and stop are ficticous and this prompt pulse is not actually present in the raw data from SNS
prompt_start = 4000.0 * sc.Unit('us')
prompt_stop = 5000.0 * sc.Unit('us')
prompt_tof_edges = spec.coords['tof']
prompt_edges = sc.concatenate(prompt_start, prompt_stop, 'tof')
prompt_tof_edges = sc.sort(sc.concatenate(prompt_tof_edges, prompt_edges, 'tof'), 'tof')
prompt = sc.DataArray(data=sc.Variable(dims=['tof'], values=[0,160000,0], unit='counts'),
            coords={'tof':prompt_tof_edges})
tof_edges = sc.linspace(dim='tof', start=18.0, stop=17000, num=100, unit='us')
spec_hist = sc.histogram(da.bins.concatenate('spectrum'), bins=tof_edges)
sc.plot({'spec':spec_hist, 'prompt':prompt})

We now want to mask out the prompt-pulse, i.e., the peak with exponential falloff inside the region where `prompt` in the figure above is nonzero.

We can do so by checking (for every event) whether the time-of-flight is within the region covered by the prompt-pulse.
As above, we first consider only a single spectrum.
The result can be stored as a new mask:

In [ ]:
spec1 = da['spectrum', 450].copy()  # copy since we do some modifications below
event_tof = spec.bins.coords['tof']
spec1.bins.masks['prompt_pulse'] = (prompt_start <= event_tof) & (event_tof < prompt_stop)
spec1.plot()

The table representation (`sc.table`) and `sc.show` illustrate this process of masking:

In [ ]:
sc.table(spec1.values[0]['event',:5])
sc.show(spec1)

We have added a new column to the event table, defining *for every event* whether it is masked or not.

The generally recommended solution is different though.
Rather than masking individual events, let us simply "sort" the events dependig on whether there fall below, inside, or above the region of the prompt-pulse.
We do not actually need to fully sort the events but rather use a *binning* proceedure, using `sc.bin`:

In [ ]:
spec2 = da['spectrum', 450].copy()  # copy since we do some modifications below
spec2 = sc.bin(spec2, edges=[prompt_tof_edges])
prompt_mask = sc.array(dims=spec2.dims, values=[False, True, False])
spec2.masks['prompt_pulse'] = prompt_mask
sc.show(spec2)

Compare this to the graphical representation for `spec1` above and to the figure of the prompt pulse.
The start and stop of the prompt pulse are used to cut the total time-of-flight interval into three sections (bins).
The center bin is masked:

In [ ]:
spec2.masks['prompt_pulse']

We can also plot the two options of the masked spectrum for comparison.
Note how in the second, recommended, option the mask is preserved in the plot, whereas in the first case the histogramming performed by `plot` necessarily has to apply the mask:

In [ ]:
sc.plot({'event-mask':spec1, 'bin-mask (2x)':spec2*sc.scalar(2.0)})

### Bonus question

Why did we not use a fine binning, e.g., with 1000 time-of-flight bins and mask a range of bins, similar to how it would be done for histogrammed (non-event) data?

### Solution

- This would add a lot of over overhead from handling many bins.
  If our instrument had 1.000.000 pixels we would have 1.000.000.000 bins, which comes with memory overhead but first and foremost compute overhead.

## Understanding the `bins` property

In [ ]:
import numpy as np
scale_scalar = 0.9 * sc.Unit('')
scale_tof = sc.array(dims=['tof'], values=[3,2,1.1])
scale_event = 0.4 + 0.2*sc.sin(spec2.bins.coords['tof'] / sc.scalar(1000.0, unit='us/rad'))

In [ ]:
ds = sc.Dataset()
ds['original'] = spec2.copy()
ds['scale_scalar'] = spec2 * scale_scalar
ds['scale_tof'] = spec2 * scale_tof
spec3 = spec2.copy()
spec3.bins.data *= scale_event
ds['scale_events'] = spec3
ds.plot()

### Concrete example

TODO Is the above useful without concrete examples?

## Binning by new parameters

After having masked the prompt-pulse we continue our investigation by considering the proton-charge log:

In [ ]:
proton_charge = da.attrs['proton_charge'].value
proton_charge.plot(marker='.')

In [ ]:
tmin = proton_charge.coords['time'].min()
tmax = proton_charge.coords['time'].max()
print(f'{tmin}\n{tmax}')

In [ ]:
pulse_time = sc.arange(dim='pulse_time',
                        start=tmin.value,
                        stop=tmax.value,
                        step=(tmax.value - tmin.value) / 100)
pulse_time

In [ ]:
spec = da['spectrum', 450].copy()  # copy since we do some modifications below
binned_spec = sc.bin(spec, edges=[pulse_time])
binned_spec

In [ ]:
#binned_spec2 = sc.bin(spec, edges=[tof_edges, pulse_time, ])
#binned_spec2

In [ ]:
binned_spec.plot()

In [ ]:
binned_spec.plot(resolution={'x':100, 'y':20})

In [ ]:
binned_spec['tof',0].plot()

In [ ]:
#da['detector_id', 3000].plot()
# da['tof', 0].plot(marker='.')  # bad because fixed resampling resolution in event mode
#da['tof', 0].bins.sum().plot(marker='.')

### Plotting slices of higher-dimensional data

### Exercise:

Using the same approach as for masking a time-of-flight bin in the previous section, mask the time period starting at about 16:30 where the proton charge is very low.

- Define appropriate edges for pulse time (use as few bins as possible, not the 100 pulse-time bins from the binning example above).
- Use `sc.bin` to apply the new binning.
- Set an appropriate bin mask.
- Plot the result to confirm that the mask is set and defined as expected.

Note:
In practice masking bad pulses would usually be done on a pulse-by-pulse basis.
This requires a slightly more complex approach and is beyond the scope of this introduction.

### Solution

## Higher dimensions and cuts

Adapt masks from 1-d case to this?

In [ ]:
da2 = sc.bin(da, edges=[prompt_tof_edges])
da2.masks['prompt_pulse'] = prompt_mask
da2.plot()

In [ ]:
import scippneutron as scn
# Note how dspacing gets unaligned if done in reverse order
#da2 = sc.bin(da, edges=[tof_edges, pulse_time, ])
#da2 = scn.convert(da2, 'tof', 'dspacing', scatter=True)
da2 = scn.convert(da2, 'tof', 'dspacing', scatter=True)

dspacing = sc.linspace(dim='dspacing', unit='Angstrom', start=0.0, stop=3.0, num=4)
# Note: Do not use 100 pulse-time bins here, extremely slow concatenate below would ensue
pulse_time = sc.arange(dim='pulse_time', start=tmin.value, stop=tmax.value, step=(tmax.value - tmin.value) / 10)
da2 = sc.bin(da2, edges=[dspacing, pulse_time, ])
#da2 = sc.bin(da2, edges=[pulse_time, ])
da2

In [ ]:
# TODO runs out of memory?
#da2.plot()

In [ ]:
da2.bins.concatenate('spectrum').plot()

In [ ]:
tmp = da2.bins.concatenate('spectrum')
lines = {}
lines['total'] = tmp.bins.concatenate('pulse_time')
for i in 0,4,9:
    lines[f'interval{i}'] = tmp['pulse_time', i]
sc.plot(lines, resolution=1000, norm='log')

In [ ]:
pulse_time = sc.arange(dim='pulse_time', start=tmin.value, stop=tmax.value, step=(tmax.value - tmin.value) / 20)
split = sc.bin(tmp, edges=[pulse_time])
sc.plot(sc.collapse(split, keep='dspacing'), resolution=5000)

In [ ]:
two_theta_bins = sc.linspace(dim='two_theta', unit='rad', start=0.2, stop=np.pi, num=9)
da2.coords['two_theta'] = scn.two_theta(da2)
#theta_sample = sc.groupby(da2, 'two_theta', bins=two_theta_bins).bins.concatenate('spectrum')
da2

In [ ]:
theta_sample = sc.groupby(da2, 'two_theta', bins=two_theta_bins).bins.concatenate('spectrum')
theta_sample

In [ ]:
#theta_sample.plot(resolution={'x':9, 'y':500})
theta_sample.plot(vmax=140*sc.Unit('counts'))

In [ ]:
#theta_sample['pulse_time', -1]['two_theta', 1].plot(resolution=8000)
dspacing_final = sc.linspace(dim='dspacing', unit='Angstrom', start=0.0, stop=2.0, num=2)

### Extract using `histogram`

Works only for 1-D cuts:

In [ ]:
dspacing_final = sc.linspace(dim='dspacing', unit='Angstrom', start=0.0, stop=2.0, num=8000)
sc.histogram(theta_sample['pulse_time', -1]['two_theta', 1],
             bins=dspacing_final).plot()

### Multi-dimensional cuts


In [ ]:
da

In [ ]:
#da3 = scn.convert(da3, 'tof', 'dspacing', scatter=True)

In [ ]:
#da3.bins.concatenate('spectrum').plot()
#cut3 = sc.bin(da3, edges=[dspacing_cut, pulse_time_cut], erase=['spectrum'])

In [ ]:
#cut3.plot()

Add fake sample env param to filter/bin

TODO Too complicated! Just bin by pulse time to which give the same visualization

da3.bins.coords['sample-env-param'] = da3.bins.coords['pulse_time'] - tmin

In [ ]:
two_theta_cut = sc.linspace(dim='two_theta', unit='rad', start=0.2, stop=1.0, num=2)
dspacing_cut = sc.linspace(dim='dspacing', unit='Angstrom', start=0.0, stop=2.0, num=2)
pulse_time_cut = sc.scalar(np.datetime64('2011-08-12T17:03:43.481640624')) \
    + sc.to_unit(sc.array(dims=['pulse_time'], unit='s', values=[0,20*60]), 'ns')

# binning two_theta works only if other event coords are also binned?
cut = sc.bin(da2, edges=[two_theta_cut, dspacing_cut, pulse_time_cut], erase=['spectrum'])

# Does not work if there is no event coord (created by `groupby` above):
#    sc.bin(theta_sample['pulse_time', -1], edges=[two_theta_cut])
# Instead:
#cut = sc.groupby(da2['pulse_time', -1], 'two_theta', bins=two_theta_cut).bins.concatenate('spectrum')
#cut = sc.groupby(cut, 'two_theta', bins=two_theta_cut).bins.concatenate('spectrum')
# But this leaves `two_theta` as a dim, would want to squeeze it
cut['two_theta', 0].plot()
#cut.plot()

In [ ]:
cut

In [ ]:
dims=['pulse']  # NXevent_data

# current
da.events.coords['pulse_time'] = sc.empty(sizes=da.events.sizes, dtype='datetime64', unit='s')
da.bins.coords['pulse_time'][...] = da.coords['pulse_time']
sc.bin(da, group=[detector_id], erase=['pulse'])

# option 1
da.bins.coords['pulse_time'] = da.coords['pulse_time']
sc.bin(da, group=[detector_id], erase=['pulse'])

# option 2 (cheaper since it avoids pass over extra column and copy)
sc.bin(da, group=[detector_id], erase=['pulse'], map_to_bins=['pulse'])
sc.bin(da, group=[detector_id], erase=['pulse'], map_erased=True)

# Unused cells